<a href="https://colab.research.google.com/github/srilamaiti/srilamaiti.github.io/blob/main/agents/multi_agent_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Libraries

In [1]:
!pip install --quiet -U langchain langchain_openai langchainhub
!!pip install --upgrade langgraph
!pip install google
!pip install wikipedia
!pip install duckduckgo-search
!pip install arxiv
!pip install python-dotenv
!pip install faiss-cpu
!pip install -U langchain-openai
!pip install langchain-experimental
!pip install langgraph-supervisor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=af9f30c42e63a990256c86850d5fad78af9dd258d371a200bd84d0073dc0ded8
  Stored in directory: /root/.cache/pip/wheels/8f/ab/cb/45ccc40522d3a1c41e1d2ad53b8f33a62f394011ec38cd71c6
Successfully built wikipedia
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 34.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.2 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6047 sha256=21ca6673a78c6f94bf8184d35381a08c89a269ee54e1313102ff78bf1138c455
  Stored in directory: /root/.cache/pip/wheels/3b/25/2a/105d6a15df6914f4d15047691c6c2

# Import Libraries

In [2]:
from typing import Annotated
from langchain.llms import OpenAI
import random
import operator
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun
from langgraph.graph.message import add_messages
from langchain.agents import AgentType
from langchain.agents import initialize_agent, Tool
from langchain.agents import tool
from typing_extensions import TypedDict
import getpass
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from IPython.display import Image, display
from dotenv import load_dotenv
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"


Mounted at /content/drive


# Setup

In [3]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env
!cat /content/drive/MyDrive/langchain_api_key.txt >> .env
!cat /content/drive/MyDrive/tavily_api_key.txt >> .env

In [4]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

In [5]:
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_PROJECT"] = "LangGraph_01"
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key

#Conversation

In [6]:
llm = OpenAI(temperature=0)
#llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

<ipython-input-6-dfe552c22ea5>:1: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


In [7]:
@tool
def product_type_search_tool(product_name: str) -> str:
    """
    Use DuckDuckGo to search for the product and summarize its type in a few words.
    """
    try:
        # Initialize DuckDuckGo search tool
        search = DuckDuckGoSearchRun()

        # Perform search
        search_results = search.invoke(f"{product_name} product name")

        # Initialize LLM for summarization
        llm = ChatOpenAI(temperature=0, model="gpt-4")

        # Summarize the search results
        summary_prompt = [
            SystemMessage(content="You are an expert summarizer. Your task is to extract and summarize product types."),
            HumanMessage(content=f"Summarize in 2-3 words the type of product for '{product_name}' based on this information:\n\n{search_results}")
        ]

        # Get summary from LLM
        product_type = llm.invoke(summary_prompt)

        return product_type.content

    except Exception as e:
        return "Unknown"


In [18]:
@tool
def critical_component_search_tool(product_type: str) -> str:
    """Use this tool to perform web searches to find out critical components of a type of product using DuckDuckGo."""
    try:

        # Initialize DuckDuckGo search tool
        search = DuckDuckGoSearchRun()

        # Perform search
        search_results = search.invoke(f"{product_type} product type")

        # Initialize LLM for summarization
        llm = ChatOpenAI(temperature=0, model="gpt-4")

        # Summarize the search results
        summary_prompt = [
            SystemMessage(content="""You are a helful assistant and an expert summarizer. Using the result from first step, please list only the unique most critical functional component names of this type of products Please do not include any specifications or compatability or preferences or impacts?"""),
            HumanMessage(content=f"What are the critical components for '{product_type}' based on this information:\n\n{search_results}")
        ]

        # Get summary from LLM
        product_type = llm.invoke(summary_prompt)

        return product_type.content

    except Exception as e:
        return "Error"

In [10]:
product_type_search_tool.invoke("CMCBL730P1")

'Cordless Brushless Blower'

In [19]:
critical_component_search_tool(product_type_search_tool.invoke("CMCBL730P1"))

'1. Lithium-ion Battery\n2. Motor\n3. Airflow System\n4. Noise Reduction System'

In [20]:
# Agentic Function: Search Product Type and Critical Components
def search_product_type_and_critical_components(product_name: str):
    """Calls both tools and returns the product type and its critical components."""
    product_type = product_type_search_tool(product_name)

    if not product_type or product_type.lower() in ["unknown", "none"]:
        return f"Could not determine the product type for '{product_name}'."

    critical_components = critical_component_search_tool(product_type)

    return f"Product Type: {product_type}\nCritical Components: {critical_components}"

# Chatbot Loop
def chatbot():
    print("🔹 Welcome to the Product Search Chatbot! Type 'quit' to exit.")
    while True:
        user_input = input("\nEnter a product name: ").strip()

        if user_input.lower() in ["quit", "exit"]:
            print("🔹 Goodbye! Chatbot session ended.")
            break

        response = search_product_type_and_critical_components(user_input)
        print("\n" + response)

chatbot()

🔹 Welcome to the Product Search Chatbot! Type 'quit' to exit.

Enter a product name: CMCBL730P1

Product Type: Cordless Leaf Blower
Critical Components: - Nozzle
- Power Source (Battery)
- Motor
- Backpack (for certain models)

Enter a product name: CMCK497P1

Product Type: Battery String Trimmer Kit
Critical Components: 1. Battery
2. Brushless String Trimmer
3. Charger
4. Curved Shaft
5. Telescopic Shaft
6. Edger
7. Jet Fan Blower
8. Electric String Trimmer
9. Combi Switch
10. Cooling Technology

Enter a product name: CMCN618B

Product Type: Cordless Brad Nailer
Critical Components: 1. Brushless Motor
2. Magazine Capacity
3. Power Source
4. LED Lights
5. Gauge
6. Nail Size
7. Battery

Enter a product name: CMCPHT818D1

Product Type: Battery Pole Trimmer
Critical Components: 1. Powerhead
2. Cordless Pole Saw Attachment
3. Pole Hedge Trimmer Attachment
4. Battery
5. Charger
6. Telescopic Aluminum Shaft
7. Bar and Chain
8. Digital Brushless Motor
9. Electric Pruning Shears
10. Brushless 